# Importing Libraries and data

In [111]:
# Importing Libraries
import pandas as pd
import numpy as np
import seaborn as sns
from imblearn.over_sampling import SMOTENC
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline
from sklearn.impute import KNNImputer
from sklearn.feature_selection import SequentialFeatureSelector, SelectKBest, f_regression,RFE
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
import joblib

In [81]:
## To Ignore Warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [82]:
# Reading Data
df = pd.read_csv("Hotel Reservations.csv", index_col = "Booking_ID")
df.head(10)

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
Booking_ID,,,,,,,,,,,,,,,,,,
INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled
INN00006,2,0,0,2,Meal Plan 2,0,Room_Type 1,346,2018,9,13,Online,0,0,0,115.00,1,Canceled
INN00007,2,0,1,3,Meal Plan 1,0,Room_Type 1,34,2017,10,15,Online,0,0,0,107.55,1,Not_Canceled
INN00008,2,0,1,3,Meal Plan 1,0,Room_Type 4,83,2018,12,26,Online,0,0,0,105.61,1,Not_Canceled
INN00009,3,0,0,4,Meal Plan 1,0,Room_Type 1,121,2018,7,6,Offline,0,0,0,96.90,1,Not_Canceled


In [83]:
#Seperating Categorical and Numerical Columns and Storing the Target Column Name
label = 'booking_status'
categorical_columns = df.drop(label, axis = 1).select_dtypes("object").columns.to_list()
numerical_columns = df.drop(label, axis=1).select_dtypes(["float", "int"]).columns.to_list()

binary_columns = df[numerical_columns].loc[:, df.nunique() == 2].columns.to_list()
non_binary_columns = df[numerical_columns].drop(binary_columns, axis = 1).columns.to_list()
non_binary_columns

['no_of_adults',
 'no_of_children',
 'no_of_weekend_nights',
 'no_of_week_nights',
 'lead_time',
 'arrival_month',
 'arrival_date',
 'no_of_previous_cancellations',
 'no_of_previous_bookings_not_canceled',
 'avg_price_per_room',
 'no_of_special_requests']

In [84]:
#Encoding the Target Column
df[label] = df[label].map({"Canceled" : 1, "Not_Canceled" : 0})
df[label]

Booking_ID
INN00001    0
INN00002    0
INN00003    1
INN00004    1
INN00005    1
           ..
INN36271    0
INN36272    1
INN36273    0
INN36274    1
INN36275    0
Name: booking_status, Length: 36275, dtype: int64

In [85]:
## Removing Zero Values from the Price Column
df.loc[df["avg_price_per_room"] == 0, "avg_price_per_room"] = np.nan

In [86]:
# Replacing the Removed Values
imputer = KNNImputer()
df["avg_price_per_room"] = imputer.fit_transform(df[["avg_price_per_room"]])

In [87]:
#checking if there is are days that don't match the month
month_group_2 = [4, 6, 9, 11]
month_group_3 = [2]
df[(df["arrival_month"].isin(month_group_3)) & (df["arrival_date"] >= 30)]


,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
Booking_ID,,,,,,,,,,,,,,,,,,


# Feature Engineering

In [88]:
#Total Number of Members for each Group
df["no_of_members"] = df["no_of_adults"] + df["no_of_children"]
#Total Number of Nights They will Stay
df["no_of_nights"] = df["no_of_weekend_nights"] + df["no_of_week_nights"]

# Pipeline

In [89]:
#scaler = PowerTransformer(method = "box-cox", standardize = False)
X = df.drop(label, axis = 1)
y = df[label]

In [90]:
#Classification Models
models = [
    ('Logistic Regression', LogisticRegression(random_state = 42)),
    ('DTC', DecisionTreeClassifier(random_state = 42)),
    ('KNC', KNeighborsClassifier()),
    ('RFC', RandomForestClassifier(random_state = 42)),
    ("XGC", XGBClassifier(random_state = 42))
]

In [91]:
#Feature Selection Object
RFE_selector = RFE(XGBRegressor(), n_features_to_select=15)

In [92]:
#Column Transformation Object
preprocessor = ColumnTransformer(transformers = [
    ("nbin", PowerTransformer(), non_binary_columns),
    ("bin", RobustScaler(), binary_columns),
    ("cat", OneHotEncoder(sparse = False, drop = "first"), categorical_columns),
])

In [93]:
# Classification Scoring Methods for Cross Validation
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score),
    'roc_auc': make_scorer(roc_auc_score)
}

In [96]:
# looping Each Model to choose the best 
for model_name, model in models:
  steps = []
  #Oversampling Step
  steps.append(("smote", SMOTENC(categorical_features=categorical_columns, random_state=42)))
  #Column Transformation Step
  steps.append(("preprocessor", preprocessor))
  #Feature Selection Step
  steps.append(("RFE_Selector", RFE_selector))
  #Model Training Step
  steps.append((model_name, model))
  #Defining Pipeline
  pipeline = Pipeline(steps = steps)
  #Evaluation Using Cross Validation
  cv_results = cross_validate(pipeline, X, y, scoring=scoring, cv=5, return_train_score=True)

  #Evaluating Based on Accuracy
  print("test :", cv_results['test_accuracy'].mean())
  print("train:", cv_results['train_accuracy'].mean())
  print('*' * 50)

test : 0.7785527222605101
train: 0.7811784975878704
**************************************************
test : 0.8634321157822191
train: 0.9932460372157135
**************************************************
test : 0.848380427291523
train: 0.8939145416953824
**************************************************
test : 0.8943073742246727
train: 0.9931082012405238
**************************************************
test : 0.8828669882839421
train: 0.9073673328738799
**************************************************


# Hyperparameter Tuning

In [104]:
# Define your parameter grid for GridSearchCV
param_grid = {
    'XGC__n_estimators': [50, 100, 200],
    'XGC__max_depth': [3, 4, 5],
    'XGC__learning_rate': [0.1, 0.2, 0.3],
    "XGC__random_state" : [42]
}

In [101]:
#XGBClassifier Pipeline
steps = []
steps.append(("smote", SMOTENC(categorical_features= categorical_columns, random_state = 42)))
steps.append(("preprocessor", preprocessor))
steps.append(("RFE_selector", RFE_selector))
steps.append(("XGC", XGBClassifier(random_state=42)))

pipeline = Pipeline(steps = steps)
cv_results = cross_validate(pipeline, X, y, scoring=scoring, cv=5, return_train_score=True)
print("test :", cv_results['test_accuracy'].mean())
print("train:", cv_results['train_accuracy'].mean())
print('*' * 50)

test : 0.8828669882839421
train: 0.9073673328738799
**************************************************


In [106]:
#Tuning the Hyperparameters using GridSearchCV
grid_search = GridSearchCV(estimator = pipeline, param_grid = param_grid, scoring=scoring["accuracy"], n_jobs=-1, return_train_score=True)

grid_search.fit(X, y)

best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

Best Parameters: {'XGC__learning_rate': 0.3, 'XGC__max_depth': 5, 'XGC__n_estimators': 200, 'XGC__random_state': 42}
Best Score: 0.8839972432804963


In [109]:
#Creating and Training the Pipeline
steps = []
steps.append(("preprocessor", preprocessor))
steps.append(("smote", SMOTE()))
steps.append(("RFE_selector", RFE_selector))
steps.append(("XGC", XGBClassifier(random_state=42, learning_rate = 0.3, max_depth = 5, n_estimators = 200)))

pipeline = Pipeline(steps=steps)
pipeline.fit(X, y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('nbin', PowerTransformer(),
                                                  ['no_of_adults',
                                                   'no_of_children',
                                                   'no_of_weekend_nights',
                                                   'no_of_week_nights',
                                                   'lead_time', 'arrival_month',
                                                   'arrival_date',
                                                   'no_of_previous_cancellations',
                                                   'no_of_previous_bookings_not_canceled',
                                                   'avg_price_per_room',
                                                   'no_of_special_requests']),
                                                 ('bin', RobustScaler(),...
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.3,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None,
                               min_child_weight=None, missing=nan,
                               monotone_constraints=None, multi_strategy=None,
                               n_estimators=200, n_jobs=None,
                               num_parallel_tree=None, random_state=42, ...))])

In [112]:
#Saving the Pipeline
joblib.dump(pipeline, "pipeline.joblib")

['pipeline.joblib']